In [1]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace  by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [2]:
import torch
import torch.nn as nn
import json
from transformers import BertTokenizer, BertModel
from sklearn.neighbors import KNeighborsClassifier

In [3]:
import json
list_sent=[]
Y_train=[]
with open('train.json', 'r') as file:
    data = file.read()
    data = json.loads(data)
    for label in data.keys():
        for sentence in data[label]:
            list_sent.append(sentence)
            Y_train.append(label)
    


In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

In [3]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [33]:
X_train=[]
for sent in list_sent:
    encoded_input = tokenizer(sent, return_tensors='pt')
    output = model(**encoded_input)
    X_train.append(output[1][0].detach().numpy())

In [37]:
# Assuming you have X_train and Y_train variables defined
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, Y_train)


KNeighborsClassifier(n_neighbors=1)

In [42]:
phrase="The role of credit scores in lending decisions is significant."
encoded_phrase = tokenizer(phrase, return_tensors='pt')
phrase_embedding = model(**encoded_phrase)[1][0].detach().numpy()
predicted_label = knn.predict([phrase_embedding])
print(predicted_label)


['Finance']


Feature based BERT

In [1]:
from transformers import BertTokenizer, BertModel
from sklearn.neighbors import KNeighborsClassifier
import json
import csv

class BertClassifFeatures():
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained("bert-base-uncased")
        self.knn = KNeighborsClassifier(n_neighbors=1)
    def train(self,train_file):
        list_sent=[]
        Y_train=[]
        with open(train_file, 'r') as file:
            data = file.read()
            data = json.loads(data)
            for label in data.keys():
                for sentence in data[label]:
                    list_sent.append(sentence)
                    Y_train.append(label)
        X_train=[]
        for sent in list_sent:
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_train.append(output[1][0].detach().numpy())
        self.knn.fit(X_train, Y_train)
    def predict(self,phrase):
        encoded_phrase = self.tokenizer(phrase, return_tensors='pt')
        phrase_embedding = self.model(**encoded_phrase)[1][0].detach().numpy()
        predicted_label = self.knn.predict([phrase_embedding])
        return predicted_label
    def create_csv(self,test_file,csv_file):
        csv_file = open(csv_file, 'w',newline='')
        writer = csv.writer(csv_file)
        writer.writerow(["ID","Label"])
        i=1
        with open(test_file, 'r') as file:
            for line in file:
                pred=self.predict(line)[0]
                writer.writerow([str(i), str(pred)])
                i+=1
        csv_file.close()
        

In [4]:
test_file='test_shuffle.txt'
csv_file='feature_based_bert.csv'	

In [2]:
bert=BertClassifFeatures()
bert.train('train.json')

In [5]:
bert.create_csv(test_file)

In [10]:
bert.predict("The Supreme Court has ruled in favor of the plaintiff in the high-profile case.")[0]

'Politics'

Finetuned BERT

In [64]:
import torch
import torch.nn as nn
import json
from transformers import BertTokenizer, BertModel

class BertClassifFinetuned():
    def __init__(self,train_file,validation_file):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained("bert-base-uncased")
        self.linear = torch.nn.Linear(768,12,bias=True)
        self.softmax = torch.nn.Softmax(dim=1)
        self.train_file=train_file
        list_sent=[]
        list_lab=[]
        with open(train_file, 'r') as file:
            data = file.read()
            data = json.loads(data)
            for label in data.keys():
                for sentence in data[label]:
                    list_sent.append(sentence)
                    list_lab.append(label)
        self.list_lab=list_lab
        self.list_sent=list_sent
        Y_train=[]
        self.dic_lab={}
        for i in range(len(list_lab)):
            if list_lab[i] not in self.dic_lab:
                self.dic_lab[list_lab[i]]=len(self.dic_lab)
            Y_train.append(self.dic_lab[list_lab[i]])
        X_train=[]
        for sent in list_sent:
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_train.append(output[1][0].detach().numpy())
        self.X_train=X_train
        self.Y_train=Y_train

        list_sent_val=[]
        list_lab_val=[]
        with open(validation_file, 'r') as file:
            data = file.read()
            data = json.loads(data)
            for label in data.keys():
                for sentence in data[label]:
                    list_sent_val.append(sentence)
                    list_lab_val.append(label)
        self.list_lab_val=list_lab_val
        self.list_sent_val=list_sent_val
        Y_val=[]
        X_val=[]
        for i,sent in enumerate(list_sent_val):
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_val.append(output[1][0].detach().numpy())
            Y_val.append(self.dic_lab[list_lab_val[i]])
        self.X_val=X_val
        self.Y_val=Y_val
    def train(self,epochs=10000):
        X_train=torch.tensor(self.X_train)
        Y_train=torch.tensor(self.Y_train)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(self.linear.parameters(), lr=0.001)
        for epoch in range(epochs):
            optimizer.zero_grad()
            outputs = self.linear(X_train)
            outputs = self.softmax(outputs)
            loss = criterion(outputs, Y_train)
            loss.backward()
            optimizer.step()
            if epoch % 1000 == 0:
                print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, loss.item()))
    def save(self,path='model.pth'):
        torch.save(self.linear.state_dict(), path)
    def load(self,path='model.pth'):
        self.linear.load_state_dict(torch.load(path))
    def predict(self,phrase):
        encoded_phrase = self.tokenizer(phrase, return_tensors='pt')
        phrase_embedding = self.model(**encoded_phrase)[1][0].detach().numpy()
        phrase_embedding=torch.tensor(phrase_embedding)
        predicted_label = self.linear(phrase_embedding)
        predicted_label=predicted_label.argmax().item()
        label_name = next(key for key, value in bert.dic_lab.items() if value == predicted_label)
        return label_name
    def validation_score(self):
        X_val=torch.tensor(self.X_val)
        Y_val=torch.tensor(self.Y_val)
        outputs = self.linear(X_val)
        outputs = self.softmax(outputs)
        predicted = outputs.argmax(1)
        correct = (predicted == Y_val).sum().item()
        accuracy = correct / len(Y_val)
        return accuracy    
    def create_csv(self,test_file,csv_file):
        csv_file = open(csv_file, 'w',newline='')
        writer = csv.writer(csv_file)
        writer.writerow(["ID","Label"])
        i=1
        with open(test_file, 'r') as file:
            for line in file:
                pred=self.predict(line)
                writer.writerow([str(i), str(pred)])
                i+=1
        csv_file.close()


In [65]:
bert=BertClassifFinetuned('train.json','validation.json')

In [90]:
bert.train(epochs=90000)

Epoch [1/90000], Loss: 1.9443
Epoch [1001/90000], Loss: 1.9439
Epoch [2001/90000], Loss: 1.9436
Epoch [3001/90000], Loss: 1.9433
Epoch [4001/90000], Loss: 1.9430
Epoch [5001/90000], Loss: 1.9427
Epoch [6001/90000], Loss: 1.9424
Epoch [7001/90000], Loss: 1.9422
Epoch [8001/90000], Loss: 1.9419
Epoch [9001/90000], Loss: 1.9416
Epoch [10001/90000], Loss: 1.9414
Epoch [11001/90000], Loss: 1.9411
Epoch [12001/90000], Loss: 1.9408
Epoch [13001/90000], Loss: 1.9406
Epoch [14001/90000], Loss: 1.9403
Epoch [15001/90000], Loss: 1.9401
Epoch [16001/90000], Loss: 1.9398
Epoch [17001/90000], Loss: 1.9396
Epoch [18001/90000], Loss: 1.9393
Epoch [19001/90000], Loss: 1.9391
Epoch [20001/90000], Loss: 1.9388
Epoch [21001/90000], Loss: 1.9385
Epoch [22001/90000], Loss: 1.9381
Epoch [23001/90000], Loss: 1.9376
Epoch [24001/90000], Loss: 1.9368
Epoch [25001/90000], Loss: 1.9355
Epoch [26001/90000], Loss: 1.9336
Epoch [27001/90000], Loss: 1.9319
Epoch [28001/90000], Loss: 1.9304
Epoch [29001/90000], Loss: 

In [91]:
bert.validation_score()

0.25

In [49]:
bert.dic_lab

{'Politics': 0,
 'Health': 1,
 'Finance': 2,
 'Travel': 3,
 'Food': 4,
 'Education': 5,
 'Environment': 6,
 'Fashion': 7,
 'Science': 8,
 'Sports': 9,
 'Technology': 10,
 'Entertainment': 11}

In [92]:
bert.save()

In [52]:
bert.load("model.pth")

In [43]:
test_file='test_shuffle.txt'
csv_file='finetuned_based_bert.csv'	

In [53]:
bert.create_csv(test_file,csv_file)

In [40]:
phrase="The robotics competition inspires students to pursue careers in STEM fields."

In [41]:
bert.predict(phrase)

'Education'

TF IDF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
td = TfidfVectorizer(max_features = 4500)
import json

list_sent=[]
Y_train=[]
with open("train.json", 'r') as file:
    data = file.read()
    data = json.loads(data)
    for label in data.keys():
        for sentence in data[label]:
            list_sent.append(sentence)
            Y_train.append(label)
X_train=[]


In [9]:
list_sent

['The mayor announced a new initiative to improve public transportation.',
 'The senator is facing criticism for her stance on the recent bill.',
 'The upcoming election has sparked intense debates among the candidates.',
 'Regular exercise and a balanced diet are key to maintaining good health.',
 'The World Health Organization has issued new guidelines on COVID-19.',
 'A new study reveals the benefits of meditation for mental health.',
 'The stock market saw a significant drop following the announcement.',
 'Investing in real estate can be a profitable venture if done correctly.',
 "The company's profits have doubled since the launch of their new product.",
 'Visiting the Grand Canyon is a breathtaking experience.',
 'The tourism industry has been severely impacted by the pandemic.',
 'Backpacking through Europe is a popular choice for young travelers.',
 'The new restaurant in town offers a fusion of Italian and Japanese cuisine.',
 'Drinking eight glasses of water a day is essentia

In [10]:
X = td.fit_transform(list_sent).toarray()

In [11]:
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.30129163, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [2]:
X_train

NameError: name 'X_train' is not defined